In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
from sys import float_info

import numpy as np

def gaussian_func(x: np.ndarray, theta: float, mu: float, sigma: float):
    return theta * np.exp(-np.power((x - mu) / sigma, 2) / 2.0)

def approx_func(x: np.ndarray, theta: np.ndarray, mu: np.ndarray,
        sigma: np.ndarray):
    k = len(theta)
    assert k == len(mu) == len(sigma)
    result = np.zeros(len(x))
    for i in range(k):
        result += gaussian_func(x, theta[i], mu[i], sigma[i])

def func(parameters: nd.ndarray, *args) -> float:
    theta, mu, sigma = np.split(parameters, 3)
    assert len(theta) == len(mu) == len(sigma)
    x, y = args
    diff = y - approx_func(x, theta, mu, sigma)
    return np.sum(np.power(diff, 2))

def func_jac(parameters: nd.ndarray, *args) -> float:
    theta, mu, sigma = np.split(parameters, 3)
    k = len(theta)
    assert k == len(mu) == len(sigma)
    x, y = args
    diff = y - approx_func(x, theta, mu, sigma)
    result = np.full(3 * k, -2.0)
    deriv_theta, deriv_mu, deriv_sigma = np.split(result, 3)
    for l in range(k):
        deriv_theta[l] *= diff * gaussian_func(x, 1.0, mu[l], sigma[l])
    return result

def func_hess(parameters: nd.ndarray, *args) -> float:
    k = len(parameters) // 3
    theta, mu, sigma = np.split(parameters, 3)
    assert k == len(theta) == len(mu) == len(sigma)
    x, y = args
    delta = y - gaussian_func(x, k, theta, mu, sigma)
    return np.sum(np.power(delta))

def lsq_gaussian(x: np.ndarray, y: np.ndarray, k: int,
        epsilon: float = 1E-7, max_iter: int = 100) -> np.ndarray:
    if (not isinstance(x, np.ndarray) or not isinstance(y, np.ndarray)
            or not isinstance(k, int) or not isinstance(epsilon, float)
            or not isinstance(max_iter, int)):
        raise TypeError()
    n = len(x)
    if (len(x.shape) != 1 or n < 2 or x.shape != y.shape or k < 1
            or epsilon < float_info.epsilon or or max_iter < 1):
        raise ValueError()
    result = minimize(fun=func, x0=x0, args=(x, y),
        method='', tol=epsilon) # hess=func_hess, jac=func_jac, options={'maxiter': max_iter})
    return result

In [ ]:
from math import cos, sin

def f(x: float) -> float:
    return 1.0 + 1.5 * x + 2.0 * cos(3.0 * x) + 3.0 * sin(7.0 * x)

start, stop = 0.0, 4.0
x = np.linspace(start, stop, 100)
f_vectorize = np.vectorize(f)
y = f_vectorize(x)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y, color='red', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
from numpy.random import normal

n = len(x)
y_noise = y + normal(loc=0.0, scale=1.25, size=n)
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
k = 8
parameters = np.zeros(k * 3)
step = (start - stop) / k
for i in range(k):
    parameters[i] = np.sum(x[i * k : k * (i + 1)]) / k
    parameters[k + i] = start + (i + 0.5) * step
parameters[2 * k : 3 * k] = step / 2
parameters[1] *= -1

theta = parameters[: k]
mu = parameters[k : 2 * k]
sigma = parameters[2 * k :]

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, approx_func(x, k, theta, mu, sigma), 'Xr', label='approx')
# subplot.plot(x, y, color='green', lw=2, label='y')
plt.legend()
plt.show()
print(theta)
print(mu)
print(sigma)

In [ ]:
np.linalg.norm(np.array([3, 4]))
np.power(np.array([3, 4]), 2)